<a href="https://colab.research.google.com/github/Ibrahim-Ezmazy/python-Projects/blob/main/VGG19_CoffeeDiseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


In [2]:
# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

# Load VGG19 model with pre-trained weights
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Define image data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Set the batch size
batch_size = 20


80134624/80134624 [==============================] - 0s 0us/step


In [4]:

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/training',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')

# Load validation data
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/validation',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')


Found 419 images belonging to 2 classes.
Found 111 images belonging to 2 classes.


In [7]:

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size)


Epoch 1/10
20/20 [==============================] - 237s 12s/step - loss: 0.1075 - accuracy: 0.9575 - val_loss: 0.0943 - val_accuracy: 0.9300
Epoch 2/10
20/20 [==============================] - 217s 11s/step - loss: 0.1194 - accuracy: 0.9574 - val_loss: 0.1084 - val_accuracy: 0.9500
Epoch 3/10
20/20 [==============================] - 253s 13s/step - loss: 0.0817 - accuracy: 0.9749 - val_loss: 0.0891 - val_accuracy: 0.9400
Epoch 4/10
20/20 [==============================] - 262s 13s/step - loss: 0.0786 - accuracy: 0.9649 - val_loss: 0.0946 - val_accuracy: 0.9400
Epoch 5/10
20/20 [==============================] - 256s 13s/step - loss: 0.0907 - accuracy: 0.9724 - val_loss: 0.1044 - val_accuracy: 0.9400
Epoch 6/10
20/20 [==============================] - 266s 13s/step - loss: 0.0854 - accuracy: 0.9724 - val_loss: 0.1043 - val_accuracy: 0.9400
Epoch 7/10
20/20 [==============================] - 255s 13s/step - loss: 0.0634 - accuracy: 0.9850 - val_loss: 0.0830 - val_accuracy: 0.9700
Epoch 

In [8]:

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print("Validation Accuracy:", val_accuracy)

# Get predictions
val_generator.reset()
y_pred = model.predict(val_generator)
y_true = val_generator.classes

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate precision, recall, and F-score
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f_score = f1_score(y_true, y_pred_classes)

print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f_score)


6/6 [==============================] - 47s 7s/step - loss: 0.0860 - accuracy: 0.9550
Validation Accuracy: 0.954954981803894
6/6 [==============================] - 47s 7s/step
Confusion Matrix:
[[26 22]
 [19 44]]
Precision: 0.6666666666666666
Recall: 0.6984126984126984
F-score: 0.6821705426356589
